In [1]:
# import modules
import kagglegym
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.regularizers import l2
from keras.optimizers import Adadelta
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing as pp
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

%matplotlib inline

Using Theano backend.


In [2]:
# Start environment
env = kagglegym.make()
observation = env.reset()
train = observation.train

In [46]:
# Data preprocessing

# https://www.kaggle.com/bguberfain/two-sigma-financial-modeling/univariate-model-with-clip/run/482189/code
# Clipped target value range to use
low_y_cut = -0.086093
high_y_cut = 0.093497

y_is_above_cut = (train.y > high_y_cut)
y_is_below_cut = (train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)

# Select the features to use
excl = ['id', 'sample', 'y', 'timestamp']
#feature_vars = [c for c in train.columns if c not in excl]
features_to_use = ['technical_30', 'technical_20', 'technical_19', 'fundamental_11']
target_var = ['y']

features = train.loc[y_is_within_cut, features_to_use]
X_train = features.values

targets = train.loc[y_is_within_cut, target_var]
y_train = targets.values

im = pp.Imputer(strategy='median')
X_train = im.fit_transform(X_train)
X_scaler = pp.RobustScaler()
X_train = X_scaler.fit_transform(X_train)
y_scaler = pp.RobustScaler()
y_train = y_scaler.fit_transform(y_train.reshape([-1,1]))

X_train = pd.DataFrame(X_train, columns=features_to_use)
y_train = pd.DataFrame(y_train, columns=target_var)
preprocess_dict = {'fillna': im, 'X_scaler': X_scaler, 'y_scaler': y_scaler}

del y_is_above_cut, y_is_below_cut, excl, targets, features

In [47]:
l2_coef=0.0001
drop_coef=0.7
shape=[len(features_to_use),256,64,32,1]
model = Sequential()
model.add(LSTM(shape[1], input_shape=(timesteps, shape[0])))
#model.add(Dense(shape[2], input_dim=shape[0], init='he_uniform', W_regularizer=l2(l2_coef)))
#model.add(Activation('relu'))
#model.add(Dropout(drop_coef))
#model.add(Dense(shape[3], init='he_uniform', W_regularizer=l2(l2_coef)))
#model.add(Activation('relu'))
#model.add(Dropout(drop_coef))
model.add(Dense(shape[4], init="he_uniform", W_regularizer=l2(l2_coef)))
model.add(Activation("linear"))
model.compile(loss="mse",optimizer="adam")


In [48]:
print("Training model")
t0 = time()
timesteps = 1
X_train_ts = pad_sequences(np.reshape(X_train.values, (X_train.values.shape[0], timesteps, X_train.values.shape[1])))
print(X_train_ts.shape)
model.fit(X_train_ts, y_train.values, nb_epoch=1,batch_size=64);
print("Done! Training time:", time() - t0)

Training model
(796880L, 1L, 11L)
Epoch 1/1
796880/796880 [==============================] - 132s - loss: 1.1281   
('Done! Training time:', 147.58500003814697)


In [38]:
print("Evaluating model on training set")
t0 = time()
m1_loss = model.evaluate(X_train_ts, y_train.values, batch_size=64, verbose=0)
print("Done! Eval time:",time() - t0)
print("Mean squared error for train dataset:",m1_loss)

Evaluating model on training set
('Done! Eval time:', 18.348999977111816)
('Mean squared error for train dataset:', 1.1251106349379048)


In [40]:
print("Predicting target on training dataset")
t0 = time()
m1_preds = model.predict(X_train_ts, batch_size=64, verbose=0)
score = r2_score(y_train, m1_preds)
print("Done! Prediction time:",time() - t0)
print("R2 score for train dataset",score)

Predicting target on training dataset
('Done! Prediction time:', 16.23300004005432)
('R2 score for train dataset', 0.0009135742732649943)


In [39]:
# Predict-step-predict routine ################################################################################
def gen_preds(model, preprocess_dict, features_to_use, print_info=True):
    env = kagglegym.make()
    # We get our initial observation by calling "reset"
    observation = env.reset()

    im = preprocess_dict['fillna']
    X_scaler = preprocess_dict['X_scaler']
    y_scaler = preprocess_dict['y_scaler']
    
    reward = 0.0
    reward_log = []
    timestamps_log = []
    pos_count = 0
    neg_count = 0

    total_pos = []
    total_neg = []

    print("Predicting")
    t0= time()
    while True:
    #    observation.features.fillna(mean_values, inplace=True)

        # Predict with model
        features_dnn = im.transform(observation.features.loc[:,features_to_use].values)
        features_dnn = X_scaler.transform(features_dnn)
        
        features_dnn_ts = pad_sequences(np.reshape(features_dnn,
                                                   (features_dnn.shape[0], timesteps, features_dnn.shape[1])))

        y_dnn = model.predict(features_dnn_ts,batch_size=32,verbose=0).clip(low_y_cut, high_y_cut)

        # Fill target df with predictions 
        observation.target.y = y_scaler.inverse_transform(y_dnn)

        observation.target.fillna(0, inplace=True)
        target = observation.target
        timestamp = observation.features["timestamp"][0]
        
        observation, reward, done, info = env.step(target)

        timestamps_log.append(timestamp)
        reward_log.append(reward)

        if (reward < 0):
            neg_count += 1
        else:
            pos_count += 1

        total_pos.append(pos_count)
        total_neg.append(neg_count)
        
        if timestamp % 100 == 0:
            if print_info:
                print("Timestamp #{}".format(timestamp))
                print("Step reward:", reward)
                print("Mean reward:", np.mean(reward_log[-timestamp:]))
                print("Positive rewards count: {0}, Negative rewards count: {1}".format(pos_count, neg_count))
                print("Positive reward %:", pos_count / (pos_count + neg_count) * 100)

            pos_count = 0
            neg_count = 0

        if done:
            break
    print("Done: %.1fs" % (time() - t0))
    print("Total reward sum:", np.sum(reward_log))
    print("Final reward mean:", np.mean(reward_log))
    print("Total positive rewards count: {0}, Total negative rewards count: {1}".format(np.sum(total_pos),
                                                                                        np.sum(total_neg)))
    print("Final positive reward %:", np.sum(total_pos) / (np.sum(total_pos) + np.sum(total_neg)) * 100)
    print(info)


In [49]:
gen_preds(model, preprocess_dict, features_to_use)

Predicting
Timestamp #1000
('Step reward:', -0.047944348833231405)
('Mean reward:', -0.13574372345434818)
Positive rewards count: 5, Negative rewards count: 90
('Positive reward %:', 0)
Timestamp #1100
('Step reward:', -0.010028695105360324)
('Mean reward:', -0.1286508996829453)
Positive rewards count: 10, Negative rewards count: 90
('Positive reward %:', 0)
Timestamp #1200
('Step reward:', -0.099286317618938399)
('Mean reward:', -0.1275602984619976)
Positive rewards count: 13, Negative rewards count: 87
('Positive reward %:', 0)
Timestamp #1300
('Step reward:', -0.083820951367486782)
('Mean reward:', -0.127840929347489)
Positive rewards count: 9, Negative rewards count: 91
('Positive reward %:', 0)
Timestamp #1400
('Step reward:', -0.15737361172250902)
('Mean reward:', -0.12777189829165031)
Positive rewards count: 9, Negative rewards count: 91
('Positive reward %:', 0)
Timestamp #1500
('Step reward:', -0.17468773830697087)
('Mean reward:', -0.12639306373567208)
Positive rewards count: